In [62]:
import requests
from bs4 import BeautifulSoup as bs

def scrape_and_parse(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:109.0) Gecko/20100101 Firefox/109.0',
        'Accept-Encoding': 'gzip, deflate, br'

    }
    html = requests.get(url , headers = headers)

    if html.status_code!=200:
        print('wrong'+url)
        return 'error'
    return bs(html.text, 'html.parser')

In [63]:
def next_page(link, soup):#翻页，返回值为下一页的soup
    try:
        key_class = ['div', 'load-more-data']
        key = soup.find(key_class[0], class_ = key_class[1])
        key = key.attrs['data-key']
        url_new = 'https://www.imdb.com/title/'+link+'/reviews/_ajax?ref_=undefined&paginationKey='+key
    except:
        return -1#到最后一页返回-1
    return scrape_and_parse(url_new)

In [64]:
def get_all_reviews(link):
    url = 'https://www.imdb.com/title/'+link+'/reviews'
    soup = scrape_and_parse(url)

    review_class = ['div', 'lister-item-content']
    title_class = ['a', 'title']
    content_class = ['div', 'text show-more__control']
    rating_class = ['span', 'rating-other-user-rating']

    each_movie_review = []

    for i in range(0, 5):
        
        if soup!=-1:
            each_soup_info = []
            reviews = soup.find_all(review_class[0], class_ = review_class[1])
            for each in reviews:
                title = each.find(title_class[0], class_ = title_class[1]).get_text()
                content = each.find(content_class[0], class_=content_class[1]).get_text()

                try:
                    rating = each.find(rating_class[0], class_=rating_class[1]).get_text().strip()
                except:
                    rating = 'null'

                each_info = [link, rating, title, content]
                each_soup_info.append(each_info)
                
            each_movie_review += each_soup_info
            soup=next_page(link, soup)
        else:
            break
    return each_movie_review

In [65]:
import pandas as pd
#尝试，无问题
links = pd.read_csv('l100.csv')['link']
all_rev = []
for i in range(0, 50):
    if i%25==0:
        print('==',i,'==')
    link = links[i]
    link = "".join(filter(str.isalnum, link))

    new_rev = get_all_reviews(link)
    all_rev += new_rev
print(len(all_rev))

== 0 ==
== 25 ==
2168


In [66]:
print(all_rev[1])

['tt0446224', 'null', ' very very boring and stupid show\n', "So,this is the one of the newest and popular Hungarian TV series? I don't think so.This show is a very good example for the Hungarian TV series.Make lot of money from something.Write an amateur story,choose some unskillful characters,and lets go:cut!Unbelievable that this type of shows(Baratok Kozt too)able to survive so many years.It's ridiculous.This show that majority of the Hungarian audience is hopeful,and in this way they are staying alive these series. It was the first comment of this show,but i think there are more people who have a same opinion like me.At the end I just want to say that,go-go Hungarian producers and writers this will be better!"]


In [67]:
for i in range(50, 100):
    if i%25==0:
        print('==',i,'==')
    link = links[i]
    link = "".join(filter(str.isalnum, link))

    new_rev = get_all_reviews(link)
    all_rev += new_rev
print(len(all_rev))

== 50 ==
== 75 ==
4919


In [68]:
all_reviews = pd.DataFrame(all_rev, columns=['link','rating', 'title', 'content'])
all_reviews.to_csv('worst_reviews.csv')

In [69]:
rep = all_reviews.groupby(['link', 'title']).size()
rep
#无重复

link       title                                              
tt0081857  ET needs some new stories!\n                           1
           ET treated Jack Hanna like a nobody.\n                 1
           First Entertainment Show and Probably the Best!!!\n    1
           Going downhill\n                                       1
           Good entertainment news show\n                         1
                                                                 ..
tt8923396  very bad !\n                                           1
           waste of a good story, terrible dialogue\n             1
           worst series ever\n                                    1
           worst tv show ive ever seen !!!!\n                     1
           😑😑😑😑\n                                                 1
Length: 4885, dtype: int64